In [100]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans

In [101]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
pd.options.mode.chained_assignment = None

In [129]:
customer_orders = pd.read_csv('./Orders - Analysis Task.csv')

In [130]:
customer_orders.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657,1312378,83290718932496,04/12/2018,2,200.0,-200.00,0.00,0.0,0.0,0.0,0,2
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686,3715657,36253792848113,01/04/2019,2,190.0,-190.00,0.00,0.0,0.0,0.0,0,2
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862,9533448,73094559597229,05/11/2018,0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2,2
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,29263220319421,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1


In [131]:
customer_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70052 entries, 0 to 70051
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           70052 non-null  object 
 1   product_type            70052 non-null  object 
 2   variant_title           70052 non-null  object 
 3   variant_sku             70052 non-null  object 
 4   variant_id              70052 non-null  int64  
 5   customer_id             70052 non-null  int64  
 6   order_id                70052 non-null  int64  
 7   day                     70052 non-null  object 
 8   net_quantity            70052 non-null  int64  
 9   gross_sales             70052 non-null  float64
 10  discounts               70052 non-null  float64
 11  returns                 70052 non-null  float64
 12  net_sales               70052 non-null  float64
 13  taxes                   70052 non-null  float64
 14  total_sales             70052 non-null

In [132]:
customer_orders.describe()

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,7.005200e+04,7.005200e+04,7.005200e+04,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000
mean,2.442320e+11,6.013091e+11,5.506075e+13,0.701179,61.776302,-4.949904,-10.246051,46.580348,9.123636,55.703982,-0.156098,0.857277
std,4.255079e+12,6.223201e+12,2.587640e+13,0.739497,31.800689,7.769972,25.154677,51.802690,10.305236,61.920557,0.369190,0.380820
min,1.001447e+07,1.000661e+06,1.000657e+13,-3.000000,0.000000,-200.000000,-237.500000,-237.500000,-47.500000,-285.000000,-3.000000,0.000000
25%,2.692223e+07,3.295695e+06,3.270317e+13,1.000000,51.670000,-8.340000,0.000000,47.080000,8.375000,56.227500,0.000000,1.000000
50%,4.494514e+07,5.566107e+06,5.522207e+13,1.000000,74.170000,0.000000,0.000000,63.330000,12.660000,76.000000,0.000000,1.000000
75%,7.743106e+07,7.815352e+06,7.736876e+13,1.000000,79.170000,0.000000,0.000000,74.170000,14.840000,89.000000,0.000000,1.000000
max,8.422212e+13,9.977409e+13,9.999554e+13,6.000000,445.000000,0.000000,0.000000,445.000000,63.340000,445.000000,0.000000,6.000000


In [133]:
customer_orders = customer_orders[customer_orders['ordered_item_quantity'] > 0 ]

In [134]:
def encode_column(column):
    if column >0:
        return 1
    if column <= 0:
        return 0
def aggregate_by_ordered_quantity(dataframe, column_list):
    aggregate_dataframe = (dataframe
                           .groupby(column_list)
                           .ordered_item_quantity.count()
                           .reset_index())
    aggregate_dataframe['products_ordered'] = (aggregate_dataframe
                                              .ordered_item_quantity
                                              .apply(encode_column))
    final_dataframe =(aggregate_dataframe
                     .groupby(column_list[0])
                     .products_ordered.sum()
                     .reset_index())
    return final_dataframe

In [135]:
customers = aggregate_by_ordered_quantity(customer_orders, ["customer_id", "product_type"])

In [136]:
print(customers.head())

   customer_id  products_ordered
0      1000661                 1
1      1001914                 1
2      1002167                 3
3      1002387                 1
4      1002419                 2


In [137]:
ordered_sum_by_customer_order = (customer_orders.groupby(['customer_id','order_id'])
                                 .ordered_item_quantity.sum()
                                 .reset_index())
returned_sum_by_customer_order = (customer_orders
                                  .groupby(['customer_id','order_id'])
                                  .returned_item_quantity.sum()
                                  .reset_index())
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order,returned_sum_by_customer_order)

In [138]:
ordered_returned_sums['average_return_rate'] = (-1*
                                    ordered_returned_sums['returned_item_quantity']/
                                    ordered_returned_sums['ordered_item_quantity']
                                                    )

In [139]:
ordered_returned_sums.head()

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
0,1000661,99119989117212,3,0,0.0
1,1001914,79758569034715,1,0,0.0
2,1002167,38156088848638,1,0,0.0
3,1002167,57440147820257,1,0,0.0
4,1002167,58825523953710,1,0,0.0


In [140]:
ordered_returned_sums[ordered_returned_sums['customer_id']==9533448]

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
39459,9533448,12837914491890,1,-1,1.0
39460,9533448,73094559597229,3,-3,1.0
39461,9533448,94864777009349,1,0,0.0


In [141]:
customer_return_rate = (ordered_returned_sums
                        .groupby('customer_id')
                        .average_return_rate
                        .mean()
                        .reset_index())

In [142]:
customer_return_rate[customer_return_rate['customer_id']==9533448]

,customer_id,average_return_rate
23276,9533448,0.666667


In [143]:
return_rates = pd.DataFrame(customer_return_rate['average_return_rate']
                           .value_counts()
                           .reset_index()
                           )
return_rates.rename(columns = 
            {'index':'average_return_rate',
             'average_return_rate':'count of unit return rate'
            },
             inplace = True
            )

In [144]:
return_rates.sort_values(by = 'average_return_rate')

,average_return_rate,count of unit return rate
0,0.000000,24823
8,0.013889,1
9,0.066667,1
11,0.083333,1
10,0.125000,1
6,0.166667,2
5,0.200000,2
4,0.250000,5
2,0.333333,13
12,0.400000,1


In [145]:
customers = pd.merge(customers,customer_return_rate,on='customer_id')

In [146]:
customers

,customer_id,products_ordered,average_return_rate
0,1000661,1,0.0
1,1001914,1,0.0
2,1002167,3,0.0
3,1002387,1,0.0
4,1002419,2,0.0
...,...,...,...
24869,97805007741979,2,0.0
24870,98854671633650,2,0.0
24871,98974226154136,1,0.0
24872,99262726332691,2,0.0


In [147]:
customers_total_spending = (customer_orders
                            .groupby('customer_id')
                            .total_sales
                            .sum()
                            .reset_index()
                             )
customers_total_spending.rename(columns = {"total_sales":'total_spending'},
                                 inplace= True)

In [148]:
customers_total_spending.head()

,customer_id,total_spending
0,1000661,260.0
1,1001914,79.2
2,1002167,234.2
3,1002387,89.0
4,1002419,103.0


In [150]:
customers.head()

,customer_id,products_ordered,average_return_rate
0,1000661,1,0.0
1,1001914,1,0.0
2,1002167,3,0.0
3,1002387,1,0.0
4,1002419,2,0.0


In [151]:
customers = customers.merge(customers_total_spending,
                            on='customer_id')

In [152]:
customers.head()

,customer_id,products_ordered,average_return_rate,total_spending
0,1000661,1,0.0,260.0
1,1001914,1,0.0,79.2
2,1002167,3,0.0,234.2
3,1002387,1,0.0,89.0
4,1002419,2,0.0,103.0


In [153]:
customers.drop(columns='customer_id',inplace=True)

In [154]:
customers.head()

,products_ordered,average_return_rate,total_spending
0,1,0.0,260.0
1,1,0.0,79.2
2,3,0.0,234.2
3,1,0.0,89.0
4,2,0.0,103.0


In [157]:
fig =  make_subplots(rows=3, cols=1,
                    subplot_titles=('Product ordered',
                                   'Average Return Rate',
                                   'Total Spending'))
fig.append_trace(go.Histogram(x=customers.products_ordered),
                row=1,col=1)
fig.append_trace(go.Histogram(x=customers.average_return_rate),
                row=2,col=1)
fig.append_trace(go.Histogram(x=customers.total_spending),
                row=3,col=1)
fig.show()

# Log transformation

In [158]:
def apply_1p_transformation(dataframe,column):
    dataframe['log_'+column]= np.log1p(dataframe[column])
    return dataframe['log_'+column]